<a href="https://colab.research.google.com/github/Rajeeb321123/Large-Language-model/blob/master/3_Load_saved_model_from_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Your input for the test

* Further more we can easily fine tunned the model for other structure of html structure. But due to lack of training dataset, I didnt do here.
Sorry, please follow the format give below in example

* Please provide the input here first before running the other code block


In [ ]:
#format of input should be like this:
input_exmaple1 = "<div class='middle'><h1>Product: HDMI Cable (6ft)</h1><p>Description: High-speed HDMI cable for 4K video and audio transmission.</p><p>Price: $8</p><img src='hdmi-cable.jpg' alt='HDMI Cable (6ft)'></div>"
input_exmaple2 = "<div class='left'><h1>Product: Wireless Keyboard and Mouse Combo</h1><p>Description: Wireless keyboard and mouse combo for clutter-free workspace.</p><p>Price: $39</p><img src='wireless-keyboard-mouse-combo.jpg' alt='Wireless Keyboard and Mouse Combo'></div>"
input_exmaple3 = "<div class='bottom'><h1>Product: Laptop Cooling Pad</h1><p>Description: Laptop cooling pad with adjustable fan speed.</p><p>Price: $25</p><img src='laptop-cooling-pad.jpg' alt='Laptop Cooling Pad'></div>"
input_exmaple4 = "<div class='left'><h1>Product: Power Strip with USB</h1><p>Description: Power strip with USB ports for charging multiple devices.</p><p>Price: $20</p><img src='power-strip-usb.jpg' alt='Power Strip with USB'></div>"
# please provide the input here first before running the other code block
# Provide your own  the input or Use the one of the example given above
your_input = "<div class='middle'><h1>Product: HDMI Cable (6ft)</h1><p>Description: High-speed HDMI cable for 4K video and audio transmission.</p><p>Price: $8</p><img src='hdmi-cable.jpg' alt='HDMI Cable (6ft)'></div>"



## Loading the Model ( my own finetune openllama 3b v2 on my own dataset)

In [ ]:
!pip install transformers
!pip install huggingfacehub
!pip install torch
!pip install bitsandbytes

ERROR: Could not find a version that satisfies the requirement huggingfacehub (from versions: none)
ERROR: No matching distribution found for huggingfacehub
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.1 MB/s eta 0:00:00


In [ ]:
import os
import torch

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)


In [ ]:
# this is basically the system prompt
# We convert every prompt like give below
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


In [ ]:
model_name = "Rajeeb321/open_llama_3b_v2-html_attribute_extraction"

In [ ]:


# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False


max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}


In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/512k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

## Main input and Inference section

In [ ]:

# Inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "parse the HTMLblock in input to identify and extract meaningful attributes relevant to e-commerce contexts (e.g., product names, prices, descriptions, images) and also identify the CSS selectors from class of html to pinpoint their location like top, bottom, left, right. Response should be in like example product:iPhone 12,description:The latest iPhone model with A14 Bionic chip,price:$799,image:iphone12.jpg,location:top",# instruction
        {your_input},# input
        "" # output, empty as model regenerate response or output part
    )
], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)

In [ ]:
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(result)

["Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nparse the HTMLblock in input to identify and extract meaningful attributes relevant to e-commerce contexts (e.g., product names, prices, descriptions, images) and also identify the CSS selectors from class of html to pinpoint their location like top, bottom, left, right. Response should be in like example product:iPhone 12,description:The latest iPhone model with A14 Bionic chip,price:$799,image:iphone12.jpg,location:top\n\n### Input:\n<div class='middle'><h1>Product: HDMI Cable (6ft)</h1><p>Description: High-speed HDMI cable for 4K video and audio transmission.</p><p>Price: $8</p><img src='hdmi-cable.jpg' alt='HDMI Cable (6ft)'></div>\n\n### Response:\nproduct:HDMI Cable (6ft),description:High-speed HDMI cable for 4K video and audio transmission.,price:$8,image:hdmi-cable.jpg,location:top\n\n### Example:\n<

In [ ]:
# Taking the response part from generated text and change it into dictionary, so it can be jsonify later on for api
def generated_text_to_dic(generated_text):
    x = generated_text[0].split('### Response:\n')

    y = x[1].split('\n\n')

    z = y[0].split(',')

    dic = {}

    for i in z:
        j = i.split(":")
        dic[j[0]] = j[1]

    return dic

In [ ]:
print(generated_text_to_dic(result))

{'product': 'HDMI Cable (6ft)', 'description': 'High-speed HDMI cable for 4K video and audio transmission.', 'price': '$8', 'image': 'hdmi-cable.jpg', 'location': 'top'}
